In [ ]:
""""
El balanceo de datos es clave para evitar que el modelo se sesgue hacia las 
clases mas frecuentes y no aprenda adecuadamente las clases menos 
representadas

Dos tecnicas principales
1. over-sampling: aumenta las imagenes de las clases menos frecuentes (duplicando)
imagenes o generando nuevas versiones a partir de las originales

2. Under-sampling: Reduce las imagenes de las clases mas frecuentes (elimina 
imagenes) al azar.

"""

In [ ]:
# Over-sampling (duplicando imágenes de clases minoritarias)

from torch.utils.data import Dataset, DataLoader
import random
import numpy as np

class OverSampleDataset(Dataset):
    def __init__(self, original_dataset, target_class_size=None):
        self.original_dataset = original_dataset
        self.target_class_size = target_class_size if target_class_size else self._get_max_class_size()
        
        self.class_indices = self._get_class_indices()
        
        # Generamos el nuevo dataset balanceado
        self.balanced_indices = self._apply_over_sampling()

    def _get_class_indices(self):
        class_indices = {i: [] for i in range(len(self.original_dataset.classes))}
        for idx, (_, label) in enumerate(self.original_dataset):
            class_indices[label].append(idx)
        return class_indices

    def _get_max_class_size(self):
        # Devuelve el tamaño de la clase mayoritaria
        return max(len(indices) for indices in self.class_indices.values())

    def _apply_over_sampling(self):
        balanced_indices = []
        for label, indices in self.class_indices.items():
            num_to_add = self.target_class_size - len(indices)
            balanced_indices.extend(indices)  # Agregamos las imágenes originales
            balanced_indices.extend(random.choices(indices, k=num_to_add))  # Duplicamos las imágenes de la clase
        return balanced_indices

    def __getitem__(self, index):
        return self.original_dataset[self.balanced_indices[index]]

    def __len__(self):
        return len(self.balanced_indices)
    
        """Explicación:

    Esta clase se basa en el dataset original (EuroSAT o el que sea que este 
    usando).

   ##  _get_class_indices() obtiene los índices de las imágenes para cada clase.

   ## _apply_over_sampling() toma las clases menos frecuentes y las "rellena" 
    para igualarlas con el tamaño de la clase mayoritaria. Esto se hace 
    duplicando imágenes de las clases minoritarias usando random.choices().
        """
    # Aquí, estamos creando un nuevo dataset a partir del train_dataset 
    # original, pero ahora balanceado por over-sampling usando la clase 
    # OverSampleDataset que implementamos antes. Este dataset 
    # sobre-sampleado aumentará las imágenes de las clases minoritarias 
    # hasta igualarlas al tamaño de las clases mayoritarias.
    over_sampled_dataset = OverSampleDataset(train_dataset)
    
    # Con esta línea, creamos un DataLoader que nos permitirá cargar el 
    # over_sampled_dataset en batches de tamaño 16. También hemos habilitado 
    # el parámetro shuffle=True para mezclar los datos en cada época (lo que 
    # ayuda a evitar sesgos durante el entrenamiento) y configurado 
    # num_workers=2 para cargar los datos en paralelo (esto acelera la carga 
    # de datos).
    over_sampled_loader = DataLoader(over_sampled_dataset, batch_size=16, shuffle=True, num_workers=2)



In [ ]:
# Under-sampling (eliminando imágenes de clases mayoritarias)

class UnderSampleDataset(Dataset):
    def __init__(self, original_dataset, target_class_size=None):
        self.original_dataset = original_dataset
        self.target_class_size = target_class_size if target_class_size else self._get_min_class_size()
        
        self.class_indices = self._get_class_indices()
        
        # Generamos el nuevo dataset balanceado
        self.balanced_indices = self._apply_under_sampling()

    def _get_class_indices(self):
        class_indices = {i: [] for i in range(len(self.original_dataset.classes))}
        for idx, (_, label) in enumerate(self.original_dataset):
            class_indices[label].append(idx)
        return class_indices

    def _get_min_class_size(self):
        # Devuelve el tamaño de la clase minoritaria
        return min(len(indices) for indices in self.class_indices.values())

    def _apply_under_sampling(self):
        balanced_indices = []
        for label, indices in self.class_indices.items():
            num_to_remove = len(indices) - self.target_class_size
            balanced_indices.extend(random.sample(indices, self.target_class_size))  # Reducimos imágenes
        return balanced_indices

    def __getitem__(self, index):
        return self.original_dataset[self.balanced_indices[index]]

    def __len__(self):
        return len(self.balanced_indices)
    
        """
    Explicación:
    Similar al OverSampleDataset, pero aquí reducimos el número de imágenes 
    de las clases mayoritarias a target_class_size (el tamaño de la clase 
    más pequeña).

    _apply_under_sampling() selecciona aleatoriamente un número de imágenes 
    para reducir el tamaño de las clases mayoritarias, utilizando 
    random.sample().
        """


In [ ]:
class UnderSampleDataset(Dataset):
    def __init__(self, original_dataset, target_class_size=None):
        self.original_dataset = original_dataset
        self.target_class_size = target_class_size if target_class_size else self._get_min_class_size()
        
        self.class_indices = self._get_class_indices()
        
        # Generamos el nuevo dataset balanceado
        self.balanced_indices = self._apply_under_sampling()

    def _get_class_indices(self):
        class_indices = {i: [] for i in range(len(self.original_dataset.classes))}
        for idx, (_, label) in enumerate(self.original_dataset):
            class_indices[label].append(idx)
        return class_indices

    def _get_min_class_size(self):
        # Devuelve el tamaño de la clase minoritaria
        return min(len(indices) for indices in self.class_indices.values())

    def _apply_under_sampling(self):
        balanced_indices = []
        for label, indices in self.class_indices.items():
            num_to_remove = len(indices) - self.target_class_size
            balanced_indices.extend(random.sample(indices, self.target_class_size))  # Reducimos imágenes
        return balanced_indices

    def __getitem__(self, index):
        return self.original_dataset[self.balanced_indices[index]]

    def __len__(self):
        return len(self.balanced_indices)


under_sampled_dataset = UnderSampleDataset(train_dataset)
under_sampled_loader = DataLoader(under_sampled_dataset, batch_size=16, shuffle=True, num_workers=2)
